In [18]:
import requests
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import os
from requests.auth import HTTPBasicAuth
import json
import time
import logging
import sys

In [19]:
import os

In [20]:
import df_construct

In [21]:
#os.chdir(r"D:\Textkernel")

In [22]:
YEAR_MONTH = '2020-08'

In [23]:
log_format = (
    '[%(asctime)s] %(levelname)-8s %(name)-12s %(message)s')

logging.basicConfig(
    level=logging.DEBUG,
    format=log_format,
    # Declare handlers
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('debug_%s.log'%YEAR_MONTH),
    ]
)
# Define your own logger name
logger = logging.getLogger("my_logger_%s"%YEAR_MONTH)

### Diese Funktion iteriert für einen gegebenen Monat über alle (bekannten) Regionen, verwendet diese als Input für die "get_month_region" Funktionen, vergleich die Anzahl der STEAs und speichert das Dataframe als JSON

In [24]:
def get_complete_month_2(month_select):
    
    
    ergebnis = DataFrame()
    #Achtung es gibt eine 17. Region (UN=Unbekannt)
    
    ##Achtung ! Eine Region ohne Einträge in einem ganzen Monat (vor aallem UN ) scheint Fehler zu verursachen 
    #region = ['BE','SL','RP','BB','MV','SH','ST','SN','HH','BW','NI','TH','UN','HB','NW','BY','HE' ]
    
    region = [ 'NW','BE','MV','SH','ST','SN','HH','NI','TH','HB','SL','HE','BB','BW','RP','BY' ]
    
    ##ACTHUnG TEST!!
    #region = ['BE','SL','BW','RP' ]
 
    

    counter = 0
    region_counter = 0
    agg2 = df_construct.aggregate_month(month_select)
    
    logger.info("Anzahl Regionen  "+str(len(region)))

    for r in region:
        # record start time
        start = time.time()
        region_counter+= 1
        
        filename = get_filename(month_select,r)
        #pfad eingefuegt
        path = 'O:/AB12/A12/QEF-STELLENANZEIGEN/_QEF-STELLENANZEIGEN/4_Daten/4_TK_data/neueDatenTest/'
        filename = os.path.join(path,filename+'.json')
        if os.path.exists(filename):
            logger.debug ('%s: file %s already exists.'%(region_counter,filename))
            continue
        logger.debug ('%s: processing %s ...'%(region_counter,filename))
            
        ##NEU
        df_construct.toobig = False
        
        #Neue Retry Funktion
        ergebnis_r = df_construct.get_month_region_retry(r,month_select)
        
        ##Neu
        
        agg1 = df_construct.aggregate_month_region(r,month_select)
        
        ##Wie viele Anzeigen runtergeladen
        counter = counter + len(ergebnis_r)
        
        ##NEU nach JSON per Monat/Region
        
        #NEU!
        #filename = get_filename(month_select,r)
        write_json(filename,ergebnis_r)
        
        ##NEU! Für den 2ten Teil des Programms muss der Filename verfügbar sein
        
        files.append(filename)
        
        ## Die Daten über die heruntergeladenen STEAS 
        mdata = []
        
        ##Monat/Jahr Region und heruntergeladene Steas
        #mdata.append(month_select)
        
        #NEU!!!
        jahr = month_select.split('-')[0]
        monat= month_select.split('-')[1]
        
        mdata.append(jahr)
        mdata.append(monat)
        
        ###
        
        mdata.append(r)
        mdata.append(len(ergebnis_r))
        
        ## Steas für diesen Monat/Jahr/Region die aktuelle Anzahl der Steas in allen Downloads und die Anzahl der Steas für
        ## den gesamten Monat des Jahres (mit den unbekannten Regionen)
        mdata.append(agg1)
        mdata.append(counter)
        mdata.append(agg2)
        
        ##Und der Dateiname für später
        mdata.append(filename)
        
        ##TEST die globale Variable für zu große Tage
        mdata.append(df_construct.toobig)
        
        ##Tupel wird der globalen Variable hinzugefügt
        metadata.append(mdata)
        
        ## Speicher wieder freigeben 
        mdata = None
        ergebnis_r = None
    
        # record execution time
        executionTime = (time.time () - start) 

        # print the difference between start
        # and end time in milli. secs
        logger.debug("The time of execution of PROCESSING df for region %s : %s"%(r, sec_to_hms(int(executionTime))))
      
    
    logger.debug("Programm fertig")    

In [25]:
##NEU Funktion soll weiter helfen die Dateinamen zu vereinheitlichen 
def get_filename(month,region):
    
    filename = "TK_"+month+"_"+region
    
    return filename

In [26]:
## Die Ausgabe ist für die json --> DB Funktion gedacht 
def get_month(filename):
    
    month = filename[3:10]
  
    return month

In [27]:
def sec_to_hms(seconds):
    h = seconds // 3600
    m = seconds % 3600 // 60
    s = seconds % 3600 % 60
    return f'{h:02d}:{m:02d}:{s:02d}'

In [28]:
##NEUE Parameter
def write_json(filename,df):
    # record start time
    start = time.time()
    
    #Das macht die Funtion nun nicht mehr selbst
    #filename = month+region
    
    
    logger.debug("Schreibe als json:  "+filename)
    print("Schreibe als json:  "+filename,file=text_file)
    
    
    ##Sollte Force_Ascii nicht true sein: NEIN für Textkernel und Co brauch man Latin-1 encoding NICHT utf-8
    df.to_json(filename+".json", orient="records", force_ascii = False)
    
    ##df.to_json(filename+".json", orient="records", force_ascii = True)
    
    logger.debug("JSON Datei erstellt")
    print("JSON Datei erstellt",file=text_file)
    
    # record execution time
    executionTime = (time.time () - start) 

    # print the difference between start
    # and end time in milli. secs
    logger.debug("The time of execution of writing on json is : %s",
          sec_to_hms(int(executionTime)))

In [29]:
##Der Februar ist der Monat mit den meisten Einträgen
### Format YYYY-MM
def testlauf():
    
    months = [YEAR_MONTH] #,'2022-05','2022-06','2022-07','2022-08','2021-06','2021-07','2021-08','2021-09','2021-10','2021-11','2021-12','2022-01','2022-02','2022-03','2022-04']
    
    for m in months:
        get_complete_month_2(m)
        


#### Startpunkt für das Progrtamm (Teil 1) , initialisiert eine Variable die den Zugriff für die Log Datei enthält sowie zwei Arrays die global vorliegen

In [ ]:
## Hier starten

##Dieser Array wird Stück für Stück befüllt auch wenn testlauf() abbricht sind die Daten nicht weg!
## Diese Variable bleibt vorläufig erhalten das sie Stück für Stück befüllt wird 

files = []

## Diese Daten müssen auch in  einem Array erfasst werden und erst dann in ein dataframe und dann json umgewandelt werden
metadata = []


##Globale Variable zur Erfassung von Tagen mit 10000+ Einträgen
df_construct.toobig = False

text_file = open(("txt_kernel_download_log_01092022_neu_skills_%s.txt"%YEAR_MONTH), "a")

#Kopie für Modul TEST
df_construct.text_file=text_file


testlauf()


print(files)

## Erstellung der JSON mit den Daten des Vorgangs
df_meta = pd.DataFrame(metadata, columns =['Jahr','Monat', 'Region','SteasDownload','SteasMonatRegion','SteasDownloadTotal','SteasMonatTotal','Dateiname','TagZuGross']) 
df_meta

write_json('testlauf1_%s'%YEAR_MONTH,df_meta)


text_file.close()

[2023-03-03 15:22:34,384] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:22:35,139] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2020-08-01__2020-08-01%7C%7C%2B1M-1d%0A%0A HTTP/1.1" 200 None
[2023-03-03 15:22:35,143] INFO     my_logger_2020-08 Anzahl Regionen  16
[2023-03-03 15:22:35,152] DEBUG    my_logger_2020-08 1: processing O:/AB12/A12/QEF-STELLENANZEIGEN/_QEF-STELLENANZEIGEN/4_Daten/4_TK_data/neueDatenTest/TK_2020-08_NW.json ...
[2023-03-03 15:22:38,170] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:22:38,820] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2020-08-01__2020-08-01%7C%7C%2B1M-1d%0A%0A&region=NW HTTP/1.1" 200 None
[2023-03-03 15:22:41,839] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:22:45,374] DEBUG    urllib3

[2023-03-03 15:27:04,098] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:27:10,505] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-08-27&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:27:17,288] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:27:22,091] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-08-28&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:27:28,395] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:27:33,008] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=NW&date=2020-08-29&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:27:39,875] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:27:42,863] DEBUG  

[2023-03-03 15:30:35,924] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-08-20&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:30:40,615] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:30:42,704] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-08-21&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:30:46,835] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:30:48,978] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-08-22&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:30:53,654] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:30:55,459] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BE&date=2020-08-23&_limit=10000&_

[2023-03-03 15:33:03,986] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:33:05,344] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-08-14&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:33:08,705] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:33:09,538] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-08-15&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:33:12,802] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:33:13,401] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=MV&date=2020-08-16&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:33:16,584] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:33:17,408] DEBUG  

[2023-03-03 15:34:58,739] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-08-07&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:35:02,225] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:35:03,536] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-08-08&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:35:06,867] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:35:07,656] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-08-09&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:35:10,854] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:35:11,986] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=SH&date=2020-08-10&_limit=10000&_

[2023-03-03 15:37:12,398] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:37:13,282] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=ST&date=2020-08-01&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:37:16,593] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:37:17,387] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=ST&date=2020-08-02&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:37:20,537] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:37:21,671] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=ST&date=2020-08-03&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:37:24,869] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:37:26,524] DEBUG  

In [14]:
text_file.close()

In [15]:

text_file = open("txt_kernel_LOG.txt", "a")

#Kopie für Modul TEST
df_construct.text_file=text_file

In [ ]:
test=  df_construct.get_month_region_retry('BW','2022-08')
print(df_construct.toobig)

[2023-03-03 15:11:44,697] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:11:45,408] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/aggregate?date__range=2022-08-01__2022-08-01%7C%7C%2B1M-1d%0A%0A&region=BW HTTP/1.1" 200 None
[2023-03-03 15:11:48,433] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:11:53,918] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-01&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:11:59,169] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15:12:07,038] DEBUG    urllib3.connectionpool https://www.jobfeed.de:443 "GET /api/v3/search?region=BW&date=2022-08-02&_limit=10000&_labels=0 HTTP/1.1" 200 None
[2023-03-03 15:12:12,570] DEBUG    urllib3.connectionpool Starting new HTTPS connection (1): www.jobfeed.de:443
[2023-03-03 15

#### Bibliotheken und globale Variablen für die Befüllung der DB 

#### Hier findet der 2.Teil statt in dem der Transfer in die DB getestet wird

In [ ]:

#x = pd.read_json("df_error.json")

x = pd.read_json("TK_2021-03_BE.json",orient="records", encoding="latin-1")

In [ ]:
x['it_skills']

In [ ]:
index = x.index
columns = x.columns
columns

In [ ]:
#x[x['it_skill_terms']=='[LabView]']
#x

In [ ]:
#x["Monat"]= x["Dateiname"].apply(lambda x: x[-5:-3]) Nicht mehr nötig

In [ ]:
files = x['Dateiname'].tolist() ##Abgeöndert

In [ ]:
files

In [ ]:
##NEUE Tests

text_file = open("txt_kernel_test_NEU.txt", "a")
#test = get_month_region_retry('2020-04','BW')

#Kopie für Modul TEST
df_construct.text_file=text_file


##Wichtig !  Region + Monat mir Tag über 10000 Einträgen
#test=  df_construct.get_month_region_retry('BW','2020-01')

#test=  df_construct.get_month_region_retry('BW','2020-04')
test=  df_construct.get_month_region_retry('SL','2018-03')

#test=  df_construct.get_month_region_retry('2020-04','BW')
#construct_day('2020-04','BE','23')

In [ ]:
test
len(test.job_id.unique())

In [ ]:
text_file = open("agg_test.txt", "a")
#test = get_month_region_retry('2020-04','BW')

#Kopie für Modul TEST
df_construct.text_file=text_file
x = df_construct.aggregate_month_region("SL","2018-03")


In [ ]:
print(x)

In [ ]:
df_input = pd.read_json('TK_2020-08_BE.json',encoding="latin-1")

In [ ]:
df_input